In [1]:
# Run the feats8 best runs with noisy data

In [1]:
import wandb
from collections import defaultdict
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from IPython import embed
import time

In [2]:
api = wandb.Api()

# Get `feats8` best runs

In [3]:
# List of tags to identify the set of sweeps in this analysis

TAGS = ["feats8"]
runs = api.runs("dhdhagar/prob-ent-resolution",
                filters={"tags": {"$in": TAGS}, 
                         "state": {"$eq": "finished"},
                         "config.pairwise_mode": {"$eq": False}
                        },
                order="-summary_metrics.best_dev_b3_f1"
               )
print(len(runs))
runs_mlp = api.runs("dhdhagar/prob-ent-resolution", 
                filters={"tags": {"$in": TAGS}, 
                         "state": {"$eq": "finished"},
                         "config.pairwise_mode": {"$eq": True}
                        },
                order="-summary_metrics.best_dev_auroc"
               )
print(len(runs_mlp))

methods = {'e2e', 'e2e-nosdp', 'frac', 'frac-nosdp', 'mlp'}
def run_key(run):
    dataset = run._attrs['config']['dataset']
    dataset_seed = run._attrs['config']['dataset_random_seed']
    method = set(run.tags).intersection(methods).pop()
    key = f"{method}_{dataset}_{dataset_seed}"
    return key

best_runs = {}
finished_runs = defaultdict(int)
all_sweeps = {}
for _runs in [runs, runs_mlp]:
    for run in tqdm(_runs):
        if len(dict(run.summary)) < 10:
            continue
        key = run_key(run)
        finished_runs[key] += 1
        if key in best_runs:
            continue
        best_runs[key] = run
        all_sweeps[key] = run.sweepName

print(f"Best runs found for {len(best_runs)} sweeps")
print(f"Total finished runs across sweeps = {sum(finished_runs.values())}")

7698
2345


  0%|          | 0/7698 [00:00<?, ?it/s]

  0%|          | 0/2345 [00:00<?, ?it/s]

Best runs found for 75 sweeps
Total finished runs across sweeps = 10040


# Run experiments

In [8]:
def launch_exp(run_key=None, model=None, dataset=None, seed=None, noise=None, gpu="cpu"):
    if run_key is not None:
        model, dataset, seed = run_key.split('_')
    else:
        run_key = f"{model}_{dataset}_{seed}"
    
    run_id = best_runs[run_key]._attrs['name']
    load_hyp_from = f"dhdhagar/prob-ent-resolution/{run_id}"
    run_tags = ','.join([f'feats8-best', f'noise_{noise}', model, dataset, f'seed_{seed}'])
    gpu_count = 0 if gpu == "cpu" else 1
    
    JOB_DESC=f"{model}_{dataset}_seed{seed}_noise{noise}"
    timestamp = !echo $(date +%s)
    JOB_NAME=f"{JOB_DESC}_{timestamp[0]}"
    job_output = !sbatch -J {JOB_NAME} -e jobs/{JOB_NAME}.err -o jobs/{JOB_NAME}.log --partition={gpu} --gres=gpu:{gpu_count} --mem=100G --time=4:00:00 \
                run_sbatch.sh e2e_scripts/train.py --skip_initial_eval --silent \
                --wandb_tags="{run_tags}" --load_hyp_from_wandb_run="{load_hyp_from}" \
                --noise_std={noise} && echo "    Logs: jobs/{JOB_NAME}.err"
    job_id = job_output[0].split()[-1]
    job_log = job_output[1].split()[-1]
    print(f'{job_id}: {job_log}')
    return job_id, job_log

In [23]:
def relaunch_failed(launched):
    for k in launched:
        state = !sacct -j {launched[k]['id']} --format="State%20"
        # First 2 rows are column headers
        if state[2].strip().split()[0] in ['FAILED', 'CRASHED', 'CANCELLED']:
            job_id, job_log = launch_exp(run_key=k, noise=launched[k]['noise'])
            launched[k] = {'id': job_id, 'noise': launched[k]['noise'], 'log': job_log}

In [ ]:
launched = {}
noise_arr = [1,2,3]
for noise in noise_arr:
    for k in best_runs:
        job_id, job_log = launch_exp(run_key=k, noise=noise)
        launched[k] = {'id': job_id, 'noise': noise, 'log': job_log}

In [ ]:
relaunch_failed(launched)

# Analyze results

In [25]:
def get_df_by_dataset(res, dataset, noise, to_latex=False):
    new_res = {}
    for _r in res:
        if dataset in _r and f'noise{noise}' in _r:
            _new_r = _r.replace(f"{dataset}_", '').replace(f"_{dataset}", '')
            _new_r = _new_r.replace(f"_noise{noise}", '').replace(f"noise{noise}_", '')
            new_res[_new_r] = res[_r]
    if to_latex:
        print(pd.DataFrame(new_res).T.style.to_latex())
    outdf = pd.DataFrame(new_res).T.sort_index()
    def highlight_max(s):
        if s.dtype == object:
            is_max = [False for _ in range(s.shape[0])]
            if '±' in s[0]:
                nums = np.array(list(map(lambda x: float(x.split('±')[0]), s)))
                is_max = nums == nums.max()
        else:
            is_max = s == s.max()
        return ['color: green' if cell else '' for cell in is_max]
    
    if outdf[outdf.keys()[0]].dtype == object:
        return outdf.style.apply(highlight_max), outdf
    return outdf.style.format('{:.2f}').apply(highlight_max), outdf

In [26]:
def get_result_dfs(best_runs):
    res_map = {
        'train_time': 'z_run_time',
        'inf_time_hac': 'z_inf_time_hac',
        'inf_time_cc': 'z_inf_time_cc',
        'inf_time_cc-nosdp': 'z_inf_time_cc-nosdp',
        'b3_f1_hac': 'best_test_b3_f1_hac',
        'b3_f1_cc': 'best_test_b3_f1_cc',
        'b3_f1_cc-fixed': 'best_test_b3_f1_cc-fixed',
        'b3_f1_cc-nosdp': 'best_test_b3_f1_cc-nosdp',
        'b3_f1_cc-nosdp-fixed': 'best_test_b3_f1_cc-nosdp-fixed',
        'vmeasure_hac': 'best_test_vmeasure_hac',
        'vmeasure_cc': 'best_test_vmeasure_cc',
        'vmeasure_cc-fixed': 'best_test_vmeasure_cc-fixed',
        'vmeasure_cc-nosdp': 'best_test_vmeasure_cc-nosdp',
        'vmeasure_cc-nosdp-fixed': 'best_test_vmeasure_cc-nosdp-fixed'
    }
    final = {}
    for run_id, run in best_runs.items():
        _key = run_id[:-2]  # Remove the seed
        if _key not in final:
            final[_key] = defaultdict(list)
        res = dict(run.summary)
        for out_key, in_key in res_map.items():
            final[_key][out_key].append(float(res[in_key]))
    means, stds, comb = {}, {}, {}
    for k in final:
        if k is not means:
            means[k] = {}
            stds[k] = {}
            comb[k] = {}
        for _k in final[k]:
            means[k][_k] = round(np.mean(final[k][_k])*(1 if 'time' in _k else 100), 2)
            stds[k][_k] = round(np.std(final[k][_k])*(1 if 'time' in _k else 100), 2)
            comb[k][_k] = f"{means[k][_k]}±{stds[k][_k]}"
    return means, stds, comb

In [29]:
# List of tags to identify the set of sweeps in this analysis

IS_TAG = "feats8-best"
IN_TAGS = ["noise_1", "noise_2", "noise_3"]
runs = api.runs("dhdhagar/prob-ent-resolution",
                filters={
                    "tags": {"$is": IS_TAG},
                    "tags": {"$in": IN_TAGS},
                    "state": {"$eq": "finished"}
                })
print(f"Runs fetched: {len(runs)}")
methods = {'e2e', 'e2e-nosdp', 'frac', 'frac-nosdp', 'mlp'}
def _run_key(run):
    dataset = run._attrs['config']['dataset']
    dataset_seed = run._attrs['config']['dataset_random_seed']
    noise = run._attrs['config']['noise_std']
    method = set(run.tags).intersection(methods).pop()
    key = f"noise{noise}_{method}_{dataset}_{dataset_seed}"
    return key
runs_dict = {_run_key(_r): _r for _r in runs}

Runs fetched: 225


In [30]:
means, stds, comb = get_result_dfs(runs_dict)

In [38]:
outdf, _outdf = get_df_by_dataset(res=comb, dataset='pubmed', noise=1)
outdf

,train_time,inf_time_hac,inf_time_cc,inf_time_cc-nosdp,b3_f1_hac,b3_f1_cc,b3_f1_cc-fixed,b3_f1_cc-nosdp,b3_f1_cc-nosdp-fixed,vmeasure_hac,vmeasure_cc,vmeasure_cc-fixed,vmeasure_cc-nosdp,vmeasure_cc-nosdp-fixed
e2e,513.0±326.4,13.0±8.74,6.0±4.34,0.4±0.49,32.78±19.02,33.52±19.97,32.04±18.14,32.56±18.92,31.64±18.01,58.48±12.59,58.62±13.51,57.84±12.68,58.26±12.92,57.88±12.72
e2e-nosdp,108.4±37.26,13.0±8.22,8.4±6.8,0.2±0.4,70.42±2.85,76.34±3.31,75.44±3.1,71.62±2.31,69.86±2.86,84.33±2.15,87.76±1.97,87.52±2.0,85.19±1.87,84.61±1.95
frac,1019.8±682.19,31.2±23.17,5.8±3.37,0.2±0.4,41.68±18.92,55.68±25.19,53.86±24.72,55.18±24.62,52.6±23.59,64.38±13.97,73.41±16.89,73.24±17.07,73.44±16.58,72.66±16.63
frac-nosdp,79.0±15.77,42.8±36.67,10.2±9.22,0.4±0.49,26.82±12.06,62.14±19.99,48.7±15.58,55.28±18.6,50.56±17.47,57.08±10.02,79.03±15.54,71.68±15.7,76.25±14.72,72.53±16.4
mlp,104.0±18.17,14.6±10.95,8.6±5.28,0.4±0.49,76.34±3.72,78.66±3.1,78.12±2.82,76.54±4.29,76.7±4.58,87.96±1.02,88.89±1.83,88.68±1.96,88.09±1.6,88.27±1.64


In [32]:
outdf, _outdf = get_df_by_dataset(res=comb, dataset='pubmed', noise=2)
outdf

,train_time,inf_time_hac,inf_time_cc,inf_time_cc-nosdp,b3_f1_hac,b3_f1_cc,b3_f1_cc-fixed,b3_f1_cc-nosdp,b3_f1_cc-nosdp-fixed,vmeasure_hac,vmeasure_cc,vmeasure_cc-fixed,vmeasure_cc-nosdp,vmeasure_cc-nosdp-fixed
e2e,550.4±227.11,70.8±96.98,7.4±4.5,0.2±0.4,31.18±11.13,41.8±16.98,36.7±14.35,34.7±13.58,37.24±15.62,57.38±6.48,65.12±11.71,61.34±10.34,60.0±9.63,61.95±11.59
e2e-nosdp,108.6±20.83,15.6±11.46,10.8±9.5,0.4±0.49,41.26±6.97,56.88±10.05,53.58±11.02,43.54±7.15,41.82±7.49,68.27±1.36,77.84±3.84,75.77±4.8,70.02±1.89,68.95±2.83
frac,680.6±394.21,68.6±73.13,8.8±8.52,0.2±0.4,33.04±15.11,42.34±22.63,35.9±17.48,36.38±19.28,36.36±17.54,59.02±8.94,65.67±15.2,61.55±12.64,62.36±12.55,61.95±13.49
frac-nosdp,87.0±23.18,90.2±66.85,12.0±11.97,0.6±0.49,33.38±10.73,38.98±10.65,34.38±8.44,32.92±8.02,34.94±8.97,59.15±7.27,71.0±12.57,69.5±11.98,68.69±11.69,70.31±12.35
mlp,103.8±11.07,14.6±10.52,11.8±14.74,0.6±0.49,43.32±11.02,52.08±17.27,46.82±16.21,43.52±14.1,39.74±13.68,68.62±4.88,75.92±8.58,75.04±7.54,70.92±7.99,71.12±8.49


In [33]:
outdf, _outdf = get_df_by_dataset(res=comb, dataset='pubmed', noise=3)
outdf

,train_time,inf_time_hac,inf_time_cc,inf_time_cc-nosdp,b3_f1_hac,b3_f1_cc,b3_f1_cc-fixed,b3_f1_cc-nosdp,b3_f1_cc-nosdp-fixed,vmeasure_hac,vmeasure_cc,vmeasure_cc-fixed,vmeasure_cc-nosdp,vmeasure_cc-nosdp-fixed
e2e,659.6±318.52,26.4±26.05,18.2±25.07,0.2±0.4,27.88±7.75,37.72±14.43,32.5±13.68,31.32±10.75,31.88±10.56,56.82±3.09,63.47±8.71,60.72±8.56,59.05±7.1,61.19±9.14
e2e-nosdp,106.8±13.01,15.0±10.3,11.4±14.15,0.2±0.4,25.9±8.66,39.04±11.67,38.14±8.73,32.04±6.08,34.5±6.53,56.64±5.77,65.3±5.23,65.16±1.17,60.26±2.17,61.81±1.95
frac,522.4±291.51,23.8±16.94,6.4±4.76,0.4±0.49,25.16±8.26,32.12±13.79,30.52±12.59,28.62±11.85,33.0±14.45,50.98±5.13,58.37±9.9,57.97±9.47,56.19±7.68,60.13±11.89
frac-nosdp,105.8±37.53,60.6±35.3,11.6±15.37,0.0±0.0,26.4±11.38,27.48±5.23,26.84±4.66,27.16±5.02,28.9±5.96,52.72±5.02,65.46±9.72,66.16±10.41,66.1±10.2,67.03±10.57
mlp,87.8±7.91,15.2±10.68,26.0±41.66,0.0±0.0,28.08±7.97,37.64±11.08,34.18±8.06,31.0±4.26,29.46±3.39,55.68±3.56,69.54±2.33,70.89±1.68,65.97±3.14,69.17±4.63


In [39]:
outdf, _outdf = get_df_by_dataset(res=comb, dataset='qian', noise=1)
outdf

,train_time,inf_time_hac,inf_time_cc,inf_time_cc-nosdp,b3_f1_hac,b3_f1_cc,b3_f1_cc-fixed,b3_f1_cc-nosdp,b3_f1_cc-nosdp-fixed,vmeasure_hac,vmeasure_cc,vmeasure_cc-fixed,vmeasure_cc-nosdp,vmeasure_cc-nosdp-fixed
e2e,978.4±487.35,35.6±7.5,6.0±5.62,0.0±0.0,67.64±7.2,66.04±9.13,62.56±15.23,65.6±10.91,64.72±12.43,83.13±5.44,82.8±5.64,82.49±6.55,82.81±6.23,83.39±6.12
e2e-nosdp,159.0±37.06,27.8±10.96,9.2±11.92,0.4±0.49,68.1±6.92,68.36±7.23,67.58±8.2,68.16±7.58,67.66±8.38,83.16±5.01,83.67±5.09,83.37±5.56,83.56±5.46,83.36±5.66
frac,817.6±210.87,80.0±96.09,5.8±7.14,0.2±0.4,68.32±6.12,69.92±6.57,69.82±6.65,69.8±6.7,69.7±6.83,83.64±4.17,84.69±4.27,84.68±4.29,84.58±4.43,84.59±4.46
frac-nosdp,107.6±24.0,118.6±108.71,5.6±6.22,0.4±0.49,68.06±5.83,68.86±6.65,67.14±6.98,68.9±6.57,68.06±7.7,83.52±4.23,84.18±4.32,82.97±5.07,84.23±4.25,83.43±5.5
mlp,93.8±22.27,121.2±166.53,5.2±5.42,0.0±0.0,68.12±7.51,51.74±19.33,50.06±20.55,51.7±19.36,50.06±20.55,83.38±5.48,79.54±6.19,79.54±6.04,79.47±6.25,79.54±6.04


In [35]:
outdf, _outdf = get_df_by_dataset(res=comb, dataset='qian', noise=2)
outdf

,train_time,inf_time_hac,inf_time_cc,inf_time_cc-nosdp,b3_f1_hac,b3_f1_cc,b3_f1_cc-fixed,b3_f1_cc-nosdp,b3_f1_cc-nosdp-fixed,vmeasure_hac,vmeasure_cc,vmeasure_cc-fixed,vmeasure_cc-nosdp,vmeasure_cc-nosdp-fixed
e2e,788.2±330.47,23.0±12.15,5.4±3.01,0.0±0.0,67.06±7.64,63.72±6.56,57.3±14.6,65.86±7.25,58.9±12.17,82.98±5.26,81.07±4.61,79.95±6.26,82.05±4.95,80.46±5.91
e2e-nosdp,155.4±32.73,26.6±12.44,5.8±5.64,0.6±0.49,68.42±7.44,67.14±6.84,63.56±7.36,66.1±7.1,62.22±8.72,83.6±5.32,82.87±5.05,81.9±4.28,82.51±5.03,80.85±5.41
frac,757.6±411.8,33.6±8.91,4.6±3.72,0.0±0.0,68.04±7.51,67.82±7.38,66.82±7.12,67.98±7.5,66.78±7.13,83.31±5.51,83.27±5.31,82.83±5.21,83.43±5.41,82.8±5.21
frac-nosdp,129.0±16.12,34.0±9.36,5.6±5.82,0.4±0.49,68.1±7.71,68.16±7.99,65.9±6.98,68.14±8.09,66.08±6.97,83.42±5.74,83.6±5.71,82.35±5.06,83.6±5.78,82.57±5.09
mlp,77.0±11.73,85.6±129.07,6.8±9.6,0.2±0.4,68.6±7.1,35.46±9.05,30.3±5.02,34.36±7.7,30.02±4.52,83.64±5.21,75.09±1.29,74.94±1.43,74.61±2.06,74.73±1.73


In [36]:
outdf, _outdf = get_df_by_dataset(res=comb, dataset='qian', noise=3)
outdf

,train_time,inf_time_hac,inf_time_cc,inf_time_cc-nosdp,b3_f1_hac,b3_f1_cc,b3_f1_cc-fixed,b3_f1_cc-nosdp,b3_f1_cc-nosdp-fixed,vmeasure_hac,vmeasure_cc,vmeasure_cc-fixed,vmeasure_cc-nosdp,vmeasure_cc-nosdp-fixed
e2e,852.2±281.64,27.2±13.36,6.8±4.87,0.0±0.0,66.86±7.38,63.52±7.34,50.96±15.99,64.24±7.49,54.06±14.51,83.01±5.29,80.93±4.75,78.27±5.18,81.33±4.92,79.31±5.45
e2e-nosdp,122.2±19.27,28.8±13.17,8.0±10.53,0.2±0.4,67.5±7.67,65.32±7.24,63.44±8.73,65.86±7.1,63.42±8.78,83.17±5.45,82.29±5.03,81.58±5.23,82.5±5.06,81.36±5.53
frac,685.0±228.79,83.6±94.84,6.2±6.05,0.0±0.0,67.18±7.52,68.36±8.07,57.76±11.54,68.0±8.02,59.02±10.86,82.99±5.14,83.52±5.55,79.41±5.78,83.38±5.53,80.43±5.5
frac-nosdp,96.6±10.98,59.2±70.06,5.6±6.25,0.0±0.0,67.48±7.52,67.94±7.92,67.98±7.9,67.94±7.92,68.0±7.87,83.19±5.43,83.57±5.78,83.6±5.77,83.6±5.73,83.61±5.74
mlp,92.0±35.06,33.8±9.39,3.8±2.4,0.2±0.4,68.22±6.91,43.3±16.86,40.34±15.98,43.0±16.93,39.82±15.98,83.51±5.15,77.77±4.62,76.98±4.88,77.45±4.77,76.78±4.99


In [37]:
outdf, _outdf = get_df_by_dataset(res=comb, dataset='arnetminer', noise=1)
outdf

,train_time,inf_time_hac,inf_time_cc,inf_time_cc-nosdp,b3_f1_hac,b3_f1_cc,b3_f1_cc-fixed,b3_f1_cc-nosdp,b3_f1_cc-nosdp-fixed,vmeasure_hac,vmeasure_cc,vmeasure_cc-fixed,vmeasure_cc-nosdp,vmeasure_cc-nosdp-fixed
e2e,553.8±120.83,44.8±28.95,45.0±68.98,1.0±2.0,61.52±9.3,54.96±12.4,53.14±12.01,55.3±12.78,52.72±12.38,70.63±7.02,71.47±5.08,70.83±4.49,71.51±5.29,70.69±4.54
e2e-nosdp,78.6±18.9,48.4±26.21,56.6±80.22,1.4±1.36,58.92±15.09,58.96±12.13,62.92±12.58,58.78±13.58,62.36±12.18,74.83±5.79,73.51±5.53,74.78±6.56,73.52±5.42,74.1±6.21
frac,672.4±275.79,96.6±102.18,58.0±89.75,1.0±1.55,60.42±11.95,54.14±8.82,48.46±6.62,52.82±9.7,48.46±7.21,72.45±7.72,71.53±6.81,69.13±4.51,70.83±6.63,69.24±4.92
frac-nosdp,75.6±16.92,32.0±15.59,46.6±48.32,2.2±1.72,63.46±10.99,61.26±12.28,59.32±14.13,61.34±12.47,58.04±15.2,76.7±6.32,76.87±6.57,76.08±6.28,76.9±6.54,75.53±6.12
mlp,143.8±42.04,31.8±12.86,37.4±48.12,1.0±0.89,56.76±15.22,56.26±8.13,53.58±9.01,54.9±9.52,52.92±9.69,76.77±5.05,75.08±5.93,74.52±5.69,74.55±5.93,74.27±5.67


In [40]:
outdf, _outdf = get_df_by_dataset(res=comb, dataset='arnetminer', noise=2)
outdf

,train_time,inf_time_hac,inf_time_cc,inf_time_cc-nosdp,b3_f1_hac,b3_f1_cc,b3_f1_cc-fixed,b3_f1_cc-nosdp,b3_f1_cc-nosdp-fixed,vmeasure_hac,vmeasure_cc,vmeasure_cc-fixed,vmeasure_cc-nosdp,vmeasure_cc-nosdp-fixed
e2e,573.0±72.26,49.6±35.61,36.6±42.63,1.0±1.55,57.02±14.84,59.68±10.76,59.36±11.53,58.74±11.79,58.44±12.37,73.84±5.23,75.18±4.92,72.18±7.33,74.03±5.96,71.62±6.96
e2e-nosdp,69.4±19.99,27.8±12.42,18.6±18.74,1.2±1.47,57.76±15.29,57.98±15.68,57.94±14.27,57.9±15.87,57.86±14.69,74.05±6.16,72.81±6.84,74.4±4.85,72.74±6.64,73.4±5.21
frac,449.0±210.47,28.8±12.42,53.6±89.93,0.6±0.8,60.5±11.59,56.92±8.53,54.02±11.77,55.92±9.06,53.6±12.41,70.26±8.29,70.41±6.64,71.64±6.0,69.85±6.8,70.23±6.66
frac-nosdp,75.8±16.77,31.8±14.44,32.0±28.62,2.2±1.72,61.88±11.48,59.08±13.6,55.5±16.57,58.56±13.94,54.88±17.11,75.46±6.42,75.57±5.92,73.41±5.47,75.31±6.02,73.2±5.45
mlp,130.2±20.55,29.8±10.96,30.4±29.28,0.6±0.49,63.52±9.61,47.96±14.37,46.62±16.67,46.5±15.35,45.22±15.88,76.38±4.73,74.58±6.87,72.7±7.01,73.73±7.34,71.1±7.58


In [41]:
outdf, _outdf = get_df_by_dataset(res=comb, dataset='arnetminer', noise=3)
outdf

,train_time,inf_time_hac,inf_time_cc,inf_time_cc-nosdp,b3_f1_hac,b3_f1_cc,b3_f1_cc-fixed,b3_f1_cc-nosdp,b3_f1_cc-nosdp-fixed,vmeasure_hac,vmeasure_cc,vmeasure_cc-fixed,vmeasure_cc-nosdp,vmeasure_cc-nosdp-fixed
e2e,749.8±53.86,53.2±37.07,52.4±54.05,0.8±0.75,56.58±15.49,58.4±11.79,55.7±13.89,58.3±12.56,55.38±14.59,72.31±6.12,71.59±6.83,72.04±4.64,71.13±7.71,71.83±5.22
e2e-nosdp,65.2±5.04,33.0±14.09,34.0±41.31,0.8±0.75,56.38±13.73,55.88±13.62,52.18±13.26,55.64±14.15,50.72±14.09,72.4±6.65,70.92±5.83,70.12±4.11,70.54±5.94,69.66±4.56
frac,618.0±159.25,101.2±91.06,39.4±50.36,0.8±0.75,58.44±13.78,56.1±9.19,55.24±9.79,55.56±9.55,55.16±9.76,72.0±6.32,69.9±6.94,69.37±7.21,69.57±7.12,69.27±7.22
frac-nosdp,56.8±6.85,33.4±14.26,33.6±33.81,1.0±0.89,60.64±11.07,56.74±15.75,56.24±15.85,56.6±15.82,55.72±16.09,74.27±6.26,74.22±6.62,74.09±6.0,74.11±6.59,73.82±6.2
mlp,113.4±11.0,32.6±13.15,22.4±18.7,0.6±0.49,62.0±10.86,48.7±10.36,45.96±8.7,48.16±10.73,45.76±8.55,74.88±6.46,70.12±4.64,70.87±5.29,69.57±5.11,70.76±5.16
